In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install -e ../.

Obtaining file:///Users/christianklaus/code/N1tram/dispatcher
  Preparing metadata (setup.py) ... done


  Attempting uninstall: dispatcher
    Found existing installation: dispatcher 1.0
    Uninstalling dispatcher-1.0:
      Successfully uninstalled dispatcher-1.0
  Running setup.py develop for dispatcher
You should consider upgrading via the '/Users/christianklaus/.pyenv/versions/3.8.12/envs/dispatcher/bin/python -m pip install --upgrade pip' command.


In [3]:
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])
import seaborn as sns
plt.style.use("dark_background")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dispatcher.data.join import JoinTables
import datetime
import math
from dispatcher.transformers.encoders import TimeFeaturesEncoder, DayFeaturesEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin, BaseEstimator

In [4]:
X_train, X_test, y_train, y_test = JoinTables().join_tables()

/Users/christianklaus/code/N1tram/dispatcher/dispatcher/data/data.py:77: DtypeWarning: Columns (4,12,22,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f'{path}/{table}.csv')


ticket features loaded
order features loaded.
carrier features loaded.


/Users/christianklaus/code/N1tram/dispatcher/dispatcher/data/data.py:77: DtypeWarning: Columns (4,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f'{path}/{table}.csv')


warehouse features loaded
Tables joined.
NAs filled. (─‿‿─)
Number of rows: 1403405
                   percent_missing
WAREHOUSE_ID                     0
RELATION_DISTANCE                0
SCR_HRS                          0
SCR_MIN                          0
PPU_DOW                          0
0.0% of rows dropped.
train test split ready. ( ◑‿◑)


In [5]:
class Debug(BaseEstimator, TransformerMixin):

    def transform(self, X):
        print(pd.DataFrame(X).head())
        print(X.shape)
        return X

    def fit(self, X, y=None, **fit_params):
        return self

In [6]:
X_train

,WAREHOUSE_ID,RELATION_DISTANCE,CARRIER_COMPANY_ID,SHOP_ID,DESTINATION_ZIP_CODE,CUSTOMER_ADDRESS_ZIP_CODE,CUSTOMER_ADDRESS_COUNTRY_ID,OC_MIN,OC_HRS,OC_DOW,OC_MONTH,PPU_MIN,PPU_HRS,PPU_DOW,SCR_MIN,SCR_HRS,SCR_DOW
1054798,673,360.317753,28,465,21241,56491,215,50,21,3,3,50,21,3,52,23,3
466417,310,817.205233,24,196,2201,8010,15,14,20,0,3,14,20,0,33,20,0
565829,613,689.355960,4,196,20018,50054,110,18,18,3,3,18,18,3,34,18,3
966598,673,360.317753,28,465,21241,6200,61,50,21,0,3,50,21,0,0,0,1
1326849,310,827.111602,13,196,2333,5440,15,3,20,0,3,3,20,0,19,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080575,673,360.317753,28,465,21241,43973,215,50,21,0,3,50,21,0,4,0,1
561279,613,689.355960,4,196,20018,16151,110,18,18,3,3,18,18,3,36,18,3
363799,310,817.205233,24,196,2201,6600,15,13,19,2,3,13,19,2,29,19,2
905822,1134,518.815866,32,664,73037,15406,209,47,13,3,3,47,13,3,47,13,3


In [7]:
 # visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

In [8]:
SimpleImputer.get_feature_names_out = (lambda self, names=None: self.feature_names_in_)

In [9]:
hr_dow = make_pipeline(TimeFeaturesEncoder(),
                      DayFeaturesEncoder()
                      )

#select = make_column_transformer(make_column_selector(dtype_exclude=datetime))

num_transformer = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())])
#max_categories > 30 for one col and > 20 for two cols take very long
cat_transformer = OneHotEncoder(max_categories=10, handle_unknown='infrequent_if_exist')

preprocessor = ColumnTransformer([
    ('num_tr', num_transformer, ['RELATION_DISTANCE']),
    ('cat_tr', cat_transformer, ['WAREHOUSE_ID', 'CARRIER_COMPANY_ID']),
    ('hr_dow', hr_dow, ['PLANNED_PICKUP_TIMESTAMP', 'CREATED_AT_SHIPMENT'])],
    remainder='drop')

pipe = make_pipeline(preprocessor, Debug(), LogisticRegression(max_iter=1000))
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_tr',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['RELATION_DISTANCE']),
                                                 ('cat_tr',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                max_categories=10),
                                                  ['WAREHOUSE_ID',
                                                   'CARRIER_COMPANY_ID']),
                                                 ('hr_dow',
                                                  Pipeline(steps=[('timefeaturesencoder',
                                                                   TimeFeaturesEncoder(hr_cols=['PLANNED_PICKUP_TIMESTAMP',
                                                                                                'CREATED_AT_SHIPMENT'])),
                                                                  ('dayfeaturesencoder',
                                                                   DayFeaturesEncoder(dow_cols=['PLANNED_PICKUP_TIMESTAMP',
                                                                                                'CREATED_AT_SHIPMENT']))]),
                                                  ['PLANNED_PICKUP_TIMESTAMP',
                                                   'CREATED_AT_SHIPMENT'])])),
                ('debug', Debug()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [10]:
# Train pipeline
pipe.fit(X_train, y_train)

# Score model 
pipe.score(X_test, y_test)

ValueError: A given column is not a column of the dataframe